In [45]:
from docx import Document
import re
from icecream import ic

In [46]:
def load_docx(file_path):
    try:
        return Document(file_path)
    except Exception as e:
        print(f"Error loading document: {e}")
        return None

doc = load_docx('data/vb_test.docx')
if not doc:
    exit()


In [47]:
import string
seps = '[.,;-+]'
ascii = string.ascii_lowercase
lowest_level = [(i + ")") for i in ascii]
levels = [lowest_level]

In [48]:
def extract_text(doc):
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                full_text.append(cell.text)
    return '\n'.join(full_text)

document_text = extract_text(doc)


In [49]:
alphabet_lst = [i for i in ascii]
digit_lst = [str(i) for i in range(99)]
bullet_levels1 = [["chương"], ["mục"], ["điều"]]
bullet_levels2 = [["chương"], ["phụ lục"], digit_lst, alphabet_lst]

In [50]:
from collections import OrderedDict

In [51]:
text = []
tree = OrderedDict()
for i, para in enumerate(doc.paragraphs):
    text.append(para.text)

In [52]:
def check_in_first3(bullet, end_bullet_idx = 3):
    for i in range(end_bullet_idx):
        if  bullet in bullet_levels1[i]:
            return True
    return False

In [53]:
full_text = []
c_check = False
tracking = False
for i, para in enumerate(text):
    if c_check == True:
        c_check = False
        continue
    if para.split(" ")[0].lower() == "chương":
        para = para + " " + text[i + 1]
        c_check = True
    else:
        c_check = False
    first_token = para.split(" ")[0]
    bullet = "###"
    if len(first_token.strip()) > 0:
        bullet = re.split(r"[.,;)]",first_token)[0]
    if check_in_first3(bullet.lower()):
        tracking = True
    else:
        tracking = False
    if (not check_in_first3(bullet.lower())) and (not tracking):
        full_text[-1] = full_text[-1] + " > " + para
        continue
    full_text.append(para)

In [54]:
from collections import OrderedDict
def check_branch_level(tree):
    level = 0
    next_tree = tree
    # print(isinstance(next_tree, OrderedDict))  # Debugging statement
    while isinstance(next_tree, OrderedDict) and next_tree:  # Check if non-empty OrderedDict
        # Get the last key in the OrderedDict
        last_key = next(reversed(next_tree))
        next_tree = next_tree[last_key]
        level += 1
    return level

In [55]:
sample_tree = OrderedDict()
# sample_tree["a"] = OrderedDict()
print(sample_tree)
print(check_branch_level(sample_tree))

OrderedDict()
0


In [56]:
def update_tree(bullet, par):
    for i in range (len(bullet_levels1)):
        if bullet.lower() in bullet_levels1[i]:
            k = tree
            current_branch_level = check_branch_level(k)
            if current_branch_level < i:
                val = OrderedDict()
                next_val = val
                for j in range (i - current_branch_level - 1):
                    next_val[""] = OrderedDict()
                    next_val = next_val[next(reversed(next_val))]
                next_val[para] = OrderedDict()
                while isinstance(k, OrderedDict) and k:  # Check if non-empty OrderedDict
                    # Get the last key in the OrderedDict
                    last_key = next(reversed(k))
                    k = k[last_key]
                k[""] = val
            else:
                for j in range(i):
                    k = k[next(reversed(k))]
                    print(k)
                k[par] = OrderedDict()

In [57]:
for i, para in enumerate(full_text):
    first_token = para.split(" ")[0]
    if len(first_token.strip()) > 0:
        bullet = re.split(r"[.,;)]",first_token)[0]
        update_tree(bullet, para)

OrderedDict([('', OrderedDict([('Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng > 1. Nghị định này quy định về Chứng chỉ hành nghề dược; kinh doanh dược; xuất khẩu, nhập khẩu thuốc; đăng ký lưu hành dược liệu, tá dược, vỏ nang; đánh giá cơ sở sản xuất thuốc tại nước ngoài; thẩm quyền, hình thức, thủ tục thu hồi nguyên liệu làm thuốc, biện pháp xử lý nguyên liệu làm thuốc bị thu hồi; hồ sơ, trình tự thủ tục và thẩm quyền cấp giấy xác nhận nội dung thông tin, quảng cáo thuốc và biện pháp quản lý giá thuốc. > 2. Nghị định này áp dụng đối với cơ quan, tổ chức, cá nhân trong nước và nước ngoài có hoạt động liên quan đến dược tại Việt Nam.', OrderedDict())]))])
OrderedDict([('Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng > 1. Nghị định này quy định về Chứng chỉ hành nghề dược; kinh doanh dược; xuất khẩu, nhập khẩu thuốc; đăng ký lưu hành dược liệu, tá dược, vỏ nang; đánh giá cơ sở sản xuất thuốc tại nước ngoài; thẩm quyền, hình thức, thủ tục thu hồi nguyên liệu làm thuốc, biện pháp xử lý 

In [58]:
def flatten_tree(tree, parent_path="", separator=" > "):
    flat_list = []
    for key, value in tree.items():
        current_path = f"{parent_path}{separator}{key}" if parent_path else key
        # print(f"Processing: {current_path}")  # Debugging step
        if value:  # Check if the value is not blank OrderedDict
            flat_list.extend(flatten_tree(value, current_path, separator))
        else:
            # If value is a blank ordered dict, convert value to blank string
            flat_list.append((current_path, ""))
    return flat_list

In [59]:
flat_data = flatten_tree(tree)
chunks = [f"{path}: {text}" for path, text in flat_data]


In [60]:
chunks

['Chương I NHỮNG QUY ĐỊNH CHUNG >  > Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng > 1. Nghị định này quy định về Chứng chỉ hành nghề dược; kinh doanh dược; xuất khẩu, nhập khẩu thuốc; đăng ký lưu hành dược liệu, tá dược, vỏ nang; đánh giá cơ sở sản xuất thuốc tại nước ngoài; thẩm quyền, hình thức, thủ tục thu hồi nguyên liệu làm thuốc, biện pháp xử lý nguyên liệu làm thuốc bị thu hồi; hồ sơ, trình tự thủ tục và thẩm quyền cấp giấy xác nhận nội dung thông tin, quảng cáo thuốc và biện pháp quản lý giá thuốc. > 2. Nghị định này áp dụng đối với cơ quan, tổ chức, cá nhân trong nước và nước ngoài có hoạt động liên quan đến dược tại Việt Nam.: ',
 'Chương I NHỮNG QUY ĐỊNH CHUNG >  > Điều 2. Giải thích từ ngữ  > Trong Nghị định này các từ, ngữ dưới đây được hiểu như sau: > 1. Thông tin thuốc là việc thu thập, cung cấp các thông tin có liên quan đến thuốc bao gồm chỉ định, chống chỉ định, liều dùng, cách dùng, phản ứng có hại của thuốc và các thông tin khác liên quan đến chất lượng, an toàn,

In [61]:
from langchain.vectorstores import FAISS
from langchain.embeddings import GPT4AllEmbeddings
from typing import List, Dict, Tuple
# Initialize the embedding model
embedding_model = GPT4AllEmbeddings(model_file="models/all-MiniLM-L6-v2-f16.gguf")

# Embed and store in FAISS
db = FAISS.from_texts(texts=chunks, embedding=embedding_model)

# Save the FAISS vector database
vector_db_path = "vectorstores/db_faiss"
db.save_local(vector_db_path)

In [74]:
import re
import json
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from icecream import ic
import pickle
# Example raw data (replace with your data)
# chunks = [
#     "Chương I NHỮNG QUY ĐỊNH CHUNG > > Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng \n 1. Nghị định này quy định về...",
#     "Chương I NHỮNG QUY ĐỊNH CHUNG > > Điều 2. Giải thích từ ngữ \n Trong Nghị định này các từ, ngữ dưới đây được hiểu như...",
#     "Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Mục 1. HỒ SƠ, THỦ TỤC CẤP, CẤP LẠI, ĐIỀU CHỈNH NỘI DUNG... > Điều 3...",
#     "Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Mục 2. ĐÀO TẠO CẬP NHẬT KIẾN THỨC CHUYÊN MÔN VỀ DƯỢC > Điều 8..."
#     # Add more chunks here
# ]

# Preprocess chunks
def preprocess_chunks(chunks):
    """
    Process raw chunks into a structured format with chapter, section, article, and content.
    """
    processed_chunks = []
    for idx, chunk in enumerate(chunks):
        # Clean up extra whitespace
        chunk = chunk.strip()
        # ic(chunk)
        # Match hierarchical parts using regex
        match = re.match(r"(Chương [^\>]+) > (Mục [^\>]+)? > (Điều [^\.\>]+)\. (.+)", chunk)
        # ic(match)
        if match:
            chapter = match.group(1).strip()
            section = match.group(2).strip() if match.group(2) else None
            article = match.group(3).strip()
            content = match.group(4).strip()
            # ic(match.group(4).strip())
        # break

            # Combine hierarchical info with content for embedding
            combined_text = f"{chapter} > {section if section else ''} > {article}. {content}".strip()

            # Append structured data
            processed_chunks.append({
                "id": idx + 1,          # Unique ID for each chunk
                "chapter": chapter,     # Chapter name
                "section": section,     # Section name (if any)
                "article": article,     # Article name
                "content": content,     # Content text
                "text": combined_text   # Full text for embedding
            })
        else:
            # Handle unmatched chunks (log for review)
            print(f"Warning: Could not process chunk: {chunk}")
    return processed_chunks

# Preprocess the chunks
processed_chunks = preprocess_chunks(chunks)
# ic(processed_chunks)
# Initialize Sentence Transformer model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Embed the processed chunks
texts = [chunk['text'] for chunk in processed_chunks]
embeddings = model.encode(texts, convert_to_numpy=True)

# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 (Euclidean) distance
index.add(embeddings)

# Save FAISS index and metadata to local
faiss.write_index(index, "faiss_index.bin")

# Save metadata to a JSON or pickle file
with open("metadata.pkl", "wb") as f:
    pickle.dump(processed_chunks, f)

print("FAISS index and metadata saved successfully!")

# Load FAISS index and metadata
def load_faiss_and_metadata(index_path, metadata_path):
    index = faiss.read_index(index_path)
    with open(metadata_path, "rb") as f:
        metadata = pickle.load(f)
    return index, metadata

# Load index and metadata
index, loaded_metadata = load_faiss_and_metadata("faiss_index.bin", "metadata.pkl")
print("FAISS index and metadata loaded successfully!")

# Query FAISS
def query_faiss(query, top_k=5):
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)
    results = []
    for idx, dist in zip(indices[0], distances[0]):
        if idx != -1:
            metadata = loaded_metadata[idx]
            metadata['score'] = dist
            results.append(metadata)
    return results

# Test the query
query_text = "Quy định chi tiết về hồ sơ đề nghị cấp Chứng chỉ hành nghề dược?"
results = query_faiss(query_text, top_k=3)

# Display the results
for result in results:
    print(result['text'])

FAISS index and metadata saved successfully!
FAISS index and metadata loaded successfully!
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Mục 1. HỒ SƠ, THỦ TỤC CẤP, CẤP LẠI, ĐIỀU CHỈNH NỘI DUNG VÀ THU HỒI CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 3. Quy định chi tiết về hồ sơ đề nghị cấp Chứng chỉ hành nghề dược > 1. Hồ sơ đề nghị cấp Chứng chỉ hành nghề dược thực hiện theo quy định tại Điều 24 của Luật dược và được quy định cụ thể như sau: > a) Đơn đề nghị cấp Chứng chỉ hành nghề dược thực hiện theo Mẫu số 02 tại Phụ lục I ban hành kèm theo Nghị định này, 02 ảnh chân dung cỡ 4 cm x 6 cm của người đề nghị cấp Chứng chỉ hành nghề dược chụp trên nền trắng trong thời gian không quá 06 tháng; > b) Bản sao có chứng thực văn bằng chuyên môn. Đối với các văn bằng do cơ sở đào tạo nước ngoài cấp, phải kèm theo bản sao có chứng thực giấy công nhận tương đương của cơ quan có thẩm quyền về công nhận tương đương theo quy định tại khoản 2 Điều 18 của Nghị định này; > c) Bản chính hoặc bản sao có chứng thực Giấy chứng 